# Data Understanding

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("data/smoker_train.csv")

## Allgemeine Analyse

In [ ]:
df.head()

In [ ]:
df.info()

## Merkmale

### Leberwerte

In [ ]:
df_urine = pd.read_csv("data/smoker_train.csv", usecols=["Urine protein"])
urine_counts = df_urine["Urine protein"].value_counts().sort_index()
print(urine_counts)